In [ ]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt


In [ ]:
# Teammate 0 code begins here

In [ ]:
# Teammate 0 code ends here

In [ ]:
# Teammate Carrie code begins here- Making Heat Maps 

In [6]:
gun_ownership_by_state = "gun_ownership_by_state.csv"
gun_ownership_by_state.head()

AttributeError: 'str' object has no attribute 'head'

In [ ]:
# Teammate Carrie code ends here

In [ ]:
# Teammate 2 code begins here

In [ ]:
# Teammate 2 code ends here

In [ ]:
# Teammate 3 code begins here

In [ ]:
# Teammate 3 code ends here